

*   Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
*   Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
*   Graficarlos.
*   Obtener conclusiones.

In [31]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO

import multiprocessing
from gensim.models import Word2Vec
from keras.preprocessing.text import text_to_word_sequence
from gensim.models.callbacks import CallbackAny2Vec

from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

from urllib import request

In [32]:
# URL The Project Gutenberg eBook of The Time Machine, by H. G. Wells
url = "https://www.gutenberg.org/files/35/35-0.txt"

# Variable para almacenar el libro
texts = []

# Comienzo y final del libro (puedes ajustar estos valores según el formato del libro en el enlace)
start = "*** START OF THIS PROJECT GUTENBERG EBOOK COSMOS ***"
end = "*** END OF THIS PROJECT GUTENBERG EBOOK COSMOS ***"

# Recuperar el contenido del libro
response = request.urlopen(url)
raw = response.read()
text = raw.decode("utf-8-sig")

# Extraer el contenido entre las secciones de inicio y final
text = text[text.find(start) + len(start):text.find(end)]

# Agregar el contenido del libro a la lista de textos
texts.append(text)

In [33]:
#Convierto la variable string en un dataframe
df = pd.read_csv(StringIO(text), sep='/n', header=None)
df.head()

<ipython-input-33-381d4eb0c6d3>:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,H. G. Wells
1,This eBook is for the use of anyone anywhere i...
2,most other parts of the world at no cost and w...
3,"whatsoever. You may copy it, give it away or r..."
4,of the Project Gutenberg License included with...


1. Preprocesamiento

In [34]:
sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

# Demos un vistazo
sentence_tokens[:10]

[['h', 'g', 'wells'],
 ['this',
  'ebook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'united',
  'states',
  'and'],
 ['most',
  'other',
  'parts',
  'of',
  'the',
  'world',
  'at',
  'no',
  'cost',
  'and',
  'with',
  'almost',
  'no',
  'restrictions'],
 ['whatsoever',
  'you',
  'may',
  'copy',
  'it',
  'give',
  'it',
  'away',
  'or',
  're',
  'use',
  'it',
  'under',
  'the',
  'terms'],
 ['of',
  'the',
  'project',
  'gutenberg',
  'license',
  'included',
  'with',
  'this',
  'ebook',
  'or',
  'online',
  'at'],
 ['www',
  'gutenberg',
  'org',
  'if',
  'you',
  'are',
  'not',
  'located',
  'in',
  'the',
  'united',
  'states',
  'you'],
 ['will',
  'have',
  'to',
  'check',
  'the',
  'laws',
  'of',
  'the',
  'country',
  'where',
  'you',
  'are',
  'located',
  'before'],
 ['using', 'this', 'ebook'],
 ['title', 'the', 'time', 'machine'],
 ['author', 'h', 'g', 'wells']]

2 - Crear los vectores (word2vec)

In [35]:
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [36]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [37]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [38]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)


Cantidad de docs en el corpus: 3093


In [39]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 933


3 - Entrenar el modelo generador

In [40]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 273250.875
Loss after epoch 1: 176282.65625
Loss after epoch 2: 175940.46875
Loss after epoch 3: 178162.3125
Loss after epoch 4: 177262.8125
Loss after epoch 5: 163673.125
Loss after epoch 6: 151947.5
Loss after epoch 7: 151498.0
Loss after epoch 8: 149773.375
Loss after epoch 9: 149076.375
Loss after epoch 10: 146905.125
Loss after epoch 11: 145136.0
Loss after epoch 12: 134105.625
Loss after epoch 13: 126401.0
Loss after epoch 14: 125224.25
Loss after epoch 15: 124085.75
Loss after epoch 16: 123375.0
Loss after epoch 17: 122413.75
Loss after epoch 18: 121619.5
Loss after epoch 19: 121554.75
Loss after epoch 20: 120563.0
Loss after epoch 21: 118897.5
Loss after epoch 22: 118825.25
Loss after epoch 23: 116990.75
Loss after epoch 24: 116614.0
Loss after epoch 25: 115259.5
Loss after epoch 26: 114737.0
Loss after epoch 27: 116215.75
Loss after epoch 28: 113962.0
Loss after epoch 29: 113166.0
Loss after epoch 30: 104776.5
Loss after epoch 31: 105601.5
Loss after epoch 

(1873688, 3601200)

4 - Ensayar

In [41]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["machine"], topn=10)

[('traveller’s', 0.40276095271110535),
 ('travelled', 0.37951239943504333),
 ('dimension', 0.37936460971832275),
 ('turf', 0.37289392948150635),
 ('table', 0.3714753985404968),
 ('time', 0.3691021502017975),
 ('“then', 0.36843374371528625),
 ('traveller', 0.36836114525794983),
 ('smiled', 0.3664730489253998),
 ('started', 0.3613404333591461)]

Vemos que las palabras que mas se relacionan con machine resultan aquellas asociadas a "viajar" o  "descripciones" de un viaje (dado que el libro trata sobre viajes en el tiempo).

In [42]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["future"], topn=10)

[('hastily', 0.0404074527323246),
 ('hung', 0.03188358247280121),
 ('seen', 0.0028626732528209686),
 ('corner', -0.006959770806133747),
 ('perhaps', -0.008442910388112068),
 ('neck', -0.009268327616155148),
 ('hail', -0.01040793489664793),
 ('as', -0.017580727115273476),
 ('further', -0.01838574931025505),
 ('lit', -0.02278711088001728)]

Estas palabras no se relacionan con futuro, no describen nada de él ni acompañan la palabra de ninguna manera.

In [43]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["species"], topn=5)

[('truth', 0.49664443731307983),
 ('hundred', 0.4960326850414276),
 ('dimensions', 0.4877045154571533),
 ('decay', 0.4811885356903076),
 ('disappeared', 0.47853463888168335)]

Volvemos a encontrar una relación que resulta mas descriptiva asociada con "species"

5 - Visualizar agrupación de vectores

In [44]:
def reduce_dimensions(model):
    num_dimensions = 2

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [45]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

En el gráfico vemos la agrupación y distribución de las palabras, las mas relacionadas forman grupos, y en este libro las palabras mas relacionadas son aquellas que funcionan como conectores o descriptores de las palabras principales que centran los grupos.